<a href="https://colab.research.google.com/github/Max-JI64/Kakao_Tech_Bootcamp/blob/main/Assignment/week7/7%EC%A3%BC%EC%B0%A8%EA%B3%BC%EC%A0%9C1_max_ji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사전 훈련된 ResNet-18

허깅페이스에서 불러온다

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from datasets import load_dataset

#예시 이미지넷 고양이 데이터
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

#이미지 전처리기 및 모델 불러오기
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

#이미지 전처리
inputs = image_processor(image, return_tensors="pt")

#예측
with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

cats_image.jpeg:   0%|          | 0.00/173k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

tabby, tabby cat


In [ ]:
print(model)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

# CIFAR-10 데이터셋

PyTorch로 CIFAR-10 불러오기

이때 ResNet-18은 224x224로 학습이 된 모델이고,   
CIFAR-10은 32x32크기이다  

따라서 CIFAR-10의 크기를 ResNet-18에 맞게 변환시켜야 한다

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor

#ResNet-18 전처리기
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
#정규화 값
imagenet_mean = processor.image_mean
imagenet_std  = processor.image_std

#이미지 전처리
transform = transforms.Compose([
    transforms.Resize(256), #이미지를 256x256으로 변환
    transforms.CenterCrop(224), #이미지를 중앙에서 224x224로 자름
    transforms.ToTensor(), #이미지를 텐서로 변환
    # CIFAR-10을 [0,1] 정규화
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

#train 데이터셋 다운로드
train_dataset = datasets.CIFAR10(
    root='./data', #다운로드 경로
    train=True,
    download=True,
    transform=transform # 전처리 함수 적용
)

#test 데이터셋 다운로드
test_dataset = datasets.CIFAR10(
    root='./data',
    train=False, #test dataset
    download=True,
    transform=transform
)

#안정적인 학습을 위해 배치 사이즈 설정 및 셔플
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
print(f"훈련 데이터셋 크기: {len(train_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")

훈련 데이터셋 크기: 50000
테스트 데이터셋 크기: 10000


# 학습 없이 바로 inference(추론)

ResNet-18 새로 불러오기 및 평가 모드 전환

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForImageClassification

#GPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#ResNet-18 불러오기
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

#분류 레드를 CIFAR-10에 맞게 10개의 클래스 분류로 수정
out_feats = model.config.hidden_sizes[-1]  # 보통 512
model.classifier = nn.Sequential(
    nn.Flatten(start_dim=1),
    nn.Linear(out_feats, 10)
)

model.to(device)
model.eval()

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

예측 및 정확도 계산

In [ ]:
total, correct = 0, 0

with torch.no_grad(): #그레디언트 계산 사용하지 않음 <- 학습을 하지 않기 때문에
    for imgs, labels in test_loader: #배치 단위로 예측 진행
        imgs, labels = imgs.to(device), labels.to(device) #이미지와 라벨을 device(GPU)로 올림
        logits = model(pixel_values=imgs).logits #예측 진행
        preds = logits.argmax(1) #가장 큰 점수의 인덱스 - 예측 클래스를 추출

        correct += (preds == labels).sum().item() #맞춘 것(예측과 라벨이 동일한것)의 합
        total   += labels.size(0) #하나의 배치 사이즈, 64를 의미

acc = correct / total * 100
print(f"파인튜닝을 하지 않았을 때 CIFAR-10 테스트 정확도: {acc:.2f}%")


파인튜닝을 하지 않았을 때 CIFAR-10 테스트 정확도: 8.74%


# I. Full Fine Tuning

기존 모델 전체의 레이어, 파라미터를 학습시킨다

In [ ]:
import os, random, math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms
from transformers import AutoImageProcessor
import numpy as np

# 재현성 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # multi-GPU 환경
    torch.backends.cudnn.deterministic = True  # 완전 재현을 원할 때
    torch.backends.cudnn.benchmark = False     # GPU 최적화 탐색 비활성 (불확실성 제거)
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"[Seed fixed to {seed}]")

## train데이터셋을 train, validation으로 분리

In [ ]:
print(len(train_dataset))

50000


In [ ]:
# CIFAR-10 라벨 매핑
id2label = {i: c for i, c in enumerate(train_dataset.dataset.classes if hasattr(train_dataset, "dataset") else train_dataset.classes)}
label2id = {v: k for k, v in id2label.items()}
id2label

#train. validation 분할
train_dataset, val_dataset = random_split(train_dataset, [40000, 10000],
                                          generator=torch.Generator().manual_seed(42))


#데이터 로더 다시 설정
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,
                          num_workers=2, pin_memory=(device.type == "cuda"), drop_last=True)
val_loader   = DataLoader(val_dataset,   batch_size=128, shuffle=False,
                          num_workers=2, pin_memory=(device.type == "cuda"))
test_loader  = DataLoader(test_dataset,  batch_size=128, shuffle=False,
                          num_workers=2, pin_memory=(device.type == "cuda"))

## ResNet-18 모델 재로드 및 출력부 헤드 설정

In [ ]:
set_seed(42)

[Seed fixed to 42]


In [ ]:
from transformers import AutoModelForImageClassification

num_labels = 10 #CIFAR-10 클래스의 개수
model = AutoModelForImageClassification.from_pretrained(
    "microsoft/resnet-18",
    num_labels=num_labels,
    ignore_mismatched_sizes=True  # 분류기 헤드 차원 재설정 시 편의용
)

#CIFAR-10 라벨 이름 반영
model.config.id2label = id2label
model.config.label2id = label2id

model.to(device) #GPU로 업로드

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-18 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([10, 512]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

## 파라미터 설정, 손실함수, 옵티마이저

In [ ]:
#풀 파인 튜닝 -> 모든 레이어를 학습, 전체 파라미터 업데이트
for p in model.parameters():
    p.requires_grad = True

criterion = nn.CrossEntropyLoss() #크로스엔트로피 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01) #Adma 옵티마이저, 학습률 0.0001, 가중치 감쇠 : 0.001 -> 과적합 방지

## 학습

In [ ]:
import time
start_time = time.time()  # 학습 시작 시간

epochs = 10 #에포크 10

for epoch in range(1, epochs + 1): #각 에포크마다 학습 진행
    model.train()  #모델 학습모드로 변환
    total, correct, loss_sum = 0, 0, 0.0 #train 이미지 수, 맞힌 개수, 손실 누적합
    for imgs, labels in train_loader: #각 배치별로
        imgs, labels = imgs.to(device), labels.to(device) #데이터를 GPU로 옮김

        optimizer.zero_grad() #이전 배치에서의 그레디언트를 초기화
        logits = model(pixel_values=imgs).logits #로짓 출력 결과값 얻음
        loss = criterion(logits, labels) #손실 계산
        loss.backward() #역전파 진행
        optimizer.step() #계산된 그레디언트로 모델의 파라미터를 한 스텝 업데이트

        loss_sum += loss.item() * labels.size(0) #이번 배치의 손실값을 누적합에 더함, lebels.size(0)는 배치의 크기
        correct += (logits.argmax(1) == labels).sum().item() #정답을 맞춘 것의 개수
        total += labels.size(0) #처리한 데이터의 개수 (배치 크기)

    #평균 손실과 정확도 계산
    train_loss = loss_sum / total
    train_acc  = correct / total * 100

    # validation 평가
    # train의 결과 평가와 같은 구조
    model.eval() #모델 평가모드로 전환
    v_total, v_correct, v_loss_sum = 0, 0, 0.0
    with torch.no_grad(): #평가에선 그레디언트를 사용하지 않음
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(pixel_values=imgs).logits

            v_loss_sum += criterion(logits, labels).item() * labels.size(0)
            v_correct += (logits.argmax(1) == labels).sum().item()
            v_total += labels.size(0)

    val_loss = v_loss_sum / v_total
    val_acc  = v_correct / v_total * 100
    #학습, 검증 손실 및 정확도 출력
    print(f"[{epoch}/{epochs}] train_loss {train_loss:.4f} acc {train_acc:.2f}% | val_loss {val_loss:.4f} acc {val_acc:.2f}%")

print("-"*30)
total_time = time.time() - start_time
print(f"Full Fine Tuning 학습 소요 시간: {total_time:.2f}초")


[1/10] train_loss 0.4217 acc 86.09% | val_loss 0.2138 acc 92.86%
[2/10] train_loss 0.1107 acc 96.63% | val_loss 0.2023 acc 93.10%
[3/10] train_loss 0.0338 acc 99.28% | val_loss 0.1917 acc 93.85%
[4/10] train_loss 0.0122 acc 99.83% | val_loss 0.1926 acc 94.16%
[5/10] train_loss 0.0049 acc 99.97% | val_loss 0.1780 acc 94.77%
[6/10] train_loss 0.0022 acc 100.00% | val_loss 0.1665 acc 95.07%
[7/10] train_loss 0.0011 acc 100.00% | val_loss 0.1697 acc 95.10%
[8/10] train_loss 0.0009 acc 100.00% | val_loss 0.1734 acc 94.93%
[9/10] train_loss 0.0067 acc 99.82% | val_loss 0.5032 acc 88.14%
[10/10] train_loss 0.1043 acc 96.36% | val_loss 0.2095 acc 93.52%
------------------------------
Full Fine Tuning 학습 소요 시간: 714.13초


## 테스트 평가

In [ ]:
#validation의 경우와 같음
model.eval()
total, correct, loss_sum = 0, 0, 0.0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(pixel_values=imgs).logits
        loss_sum += criterion(logits, labels).item() * labels.size(0)
        correct += (logits.argmax(1) == labels).sum().item()
        total += labels.size(0)

test_loss = loss_sum / total
test_acc  = correct / total * 100
print("Full Fine Tuning CIFAR-10 테스트 손실 및 정확도")
print(f"test_loss {test_loss:.4f} acc {test_acc:.2f}%")

Full Fine Tuning CIFAR-10 테스트 손실 및 정확도
test_loss 0.2336 acc 92.95%


# II. Partial Fine Tuning

기존 모델의 일부 레이어만 학습시킨다  
초기층은 그대로 유지,  
후반부 블록이나 분류기와 같은 상위 계층만 학습

이를 통해 이미 학습된 엣지, 색상과 같은 저수준의 특징 그대로 활용  
학습 속도 빠르고 자원 절약  
작은 데이터셋일때 과적합 방지

## ResNet-18 모델 재로드 및 출력부 헤드 설정

In [ ]:
set_seed(42)

[Seed fixed to 42]


In [ ]:
from transformers import AutoModelForImageClassification

num_labels = 10 #CIFAR-10 클래스의 개수
model = AutoModelForImageClassification.from_pretrained(
    "microsoft/resnet-18",
    num_labels=num_labels,
    ignore_mismatched_sizes=True  # 분류기 헤드 차원 재설정 시 편의용
)

#CIFAR-10 라벨 이름 반영
model.config.id2label = id2label
model.config.label2id = label2id

model.to(device) #GPU로 업로드

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-18 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([10, 512]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

## 일부 레이어만 학습하도록, 손실함수, 옵티마이저

In [ ]:
#전체 파라미터 동결
for p in model.parameters():
    p.requires_grad = False

#마지막 블록 layer4와 분류기만 학습 허용
for p in model.resnet.encoder.stages[-1].parameters():
    p.requires_grad = True
for p in model.classifier.parameters():
    p.requires_grad = True

criterion = nn.CrossEntropyLoss() #크로스엔트로피 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01) #Adma 옵티마이저, 학습률 0.0001, 가중치 감쇠 : 0.001 -> 과적합 방지

## 학습

In [ ]:
import time
start_time = time.time() # 학습 시작 시간

epochs = 10 #에포크 10

for epoch in range(1, epochs + 1): #각 에포크마다 학습 진행
    model.train()  #모델 학습모드로 변환
    total, correct, loss_sum = 0, 0, 0.0 #train 이미지 수, 맞힌 개수, 손실 누적합
    for imgs, labels in train_loader: #각 배치별로
        imgs, labels = imgs.to(device), labels.to(device) #데이터를 GPU로 옮김

        optimizer.zero_grad() #이전 배치에서의 그레디언트를 초기화
        logits = model(pixel_values=imgs).logits #로짓 출력 결과값 얻음
        loss = criterion(logits, labels) #손실 계산
        loss.backward() #역전파 진행
        optimizer.step() #계산된 그레디언트로 모델의 파라미터를 한 스텝 업데이트

        loss_sum += loss.item() * labels.size(0) #이번 배치의 손실값을 누적합에 더함, lebels.size(0)는 배치의 크기
        correct += (logits.argmax(1) == labels).sum().item() #정답을 맞춘 것의 개수
        total += labels.size(0) #처리한 데이터의 개수 (배치 크기)

    #평균 손실과 정확도 계산
    train_loss = loss_sum / total
    train_acc  = correct / total * 100

    # validation 평가
    # train의 결과 평가와 같은 구조
    model.eval() #모델 평가모드로 전환
    v_total, v_correct, v_loss_sum = 0, 0, 0.0
    with torch.no_grad(): #평가에선 그레디언트를 사용하지 않음
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(pixel_values=imgs).logits

            v_loss_sum += criterion(logits, labels).item() * labels.size(0)
            v_correct += (logits.argmax(1) == labels).sum().item()
            v_total += labels.size(0)

    val_loss = v_loss_sum / v_total
    val_acc  = v_correct / v_total * 100
    #학습, 검증 손실 및 정확도 출력
    print(f"[{epoch}/{epochs}] train_loss {train_loss:.4f} acc {train_acc:.2f}% | val_loss {val_loss:.4f} acc {val_acc:.2f}%")

print("-"*30)
total_time = time.time() - start_time
print(f"Pratial Fine Tuning 학습 소요 시간: {total_time:.2f}초")


[1/10] train_loss 0.5519 acc 81.41% | val_loss 0.3335 acc 88.17%
[2/10] train_loss 0.1954 acc 94.09% | val_loss 0.3080 acc 89.24%
[3/10] train_loss 0.0610 acc 98.93% | val_loss 0.3066 acc 89.77%
[4/10] train_loss 0.0185 acc 99.89% | val_loss 0.3046 acc 90.26%
[5/10] train_loss 0.0073 acc 99.99% | val_loss 0.3099 acc 90.36%
[6/10] train_loss 0.0039 acc 100.00% | val_loss 0.3120 acc 90.53%
[7/10] train_loss 0.0025 acc 100.00% | val_loss 0.3196 acc 90.53%
[8/10] train_loss 0.0020 acc 100.00% | val_loss 0.3279 acc 90.41%
[9/10] train_loss 0.0014 acc 100.00% | val_loss 0.3310 acc 90.41%
[10/10] train_loss 0.0011 acc 100.00% | val_loss 0.3371 acc 90.54%
------------------------------
Pratial Fine Tuning 학습 소요 시간: 469.51초


## 테스트 평가

In [ ]:
#validation의 경우와 같음
model.eval()
total, correct, loss_sum = 0, 0, 0.0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(pixel_values=imgs).logits
        loss_sum += criterion(logits, labels).item() * labels.size(0)
        correct += (logits.argmax(1) == labels).sum().item()
        total += labels.size(0)

test_loss = loss_sum / total
test_acc  = correct / total * 100
print("Partial Fine Tuning CIFAR-10 테스트 손실 및 정확도")
print(f"test_loss {test_loss:.4f} acc {test_acc:.2f}%")

Partial Fine Tuning CIFAR-10 테스트 손실 및 정확도
test_loss 0.3496 acc 90.56%


# III. Progressive Fine Tuning

In [ ]:
set_seed(42)

[Seed fixed to 42]


단계적 파인튜닝  
처음엔 모델의 상위레이어만 동결 해제,  
학습이 어느정도 안정되면  
학습 후 일부 하위 레이어 동결해제  

점진적으로 학습 범위 넓혀가기

처음부터 모든 파라미터를 학습하면  
기존의 사전학습 가중치가 무너질 수 있음  
특히 작은 데이터셋에선 과적합, 손실 폭주가 발생  

이것은 학습량을 서서히 늘려가기 때문에 과적합 억제 가능

## ResNet-18 모델 재로드 및 출력부 헤드 설정

In [ ]:
set_seed(42)

[Seed fixed to 42]


In [ ]:
from transformers import AutoModelForImageClassification

num_labels = 10 #CIFAR-10 클래스의 개수
model = AutoModelForImageClassification.from_pretrained(
    "microsoft/resnet-18",
    num_labels=num_labels,
    ignore_mismatched_sizes=True  # 분류기 헤드 차원 재설정 시 편의용
)

#CIFAR-10 라벨 이름 반영
model.config.id2label = id2label
model.config.label2id = label2id

model.to(device) #GPU로 업로드

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-18 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([10, 512]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

## 점진적인 레이어 학습 설정

In [ ]:
# 모든 레이어 동결
for p in model.parameters():
    p.requires_grad = False

# 분류기만 학습 허용
for p in model.classifier.parameters():
    p.requires_grad = True

## 손실함수, 옵티마이저 재구성

In [ ]:
criterion = nn.CrossEntropyLoss()

#단계적 학습을 진행하기 때문에 옵티마이저를 변환하여야 한다
#왜냐하면 새로 파라미터가 열리면 옵티마이저를 새로 생성해야 하기 때문이다
def build_optimizer():
    """현재 requires_grad=True인 파라미터를 기준으로 param group 구성"""
    #학습 가능한 파라미터
    head_params = [p for n,p in model.named_parameters() if p.requires_grad and n.startswith("classifier")]
    #학습 불가능한 파라미터 (사전 학습된 본체)
    bb_params   = [p for n,p in model.named_parameters() if p.requires_grad and not n.startswith("classifier")]
    param_groups = []
    if head_params: param_groups.append({"params": head_params, "lr": 1e-3}) #학습해야 하는건 빠르게
    if bb_params:   param_groups.append({"params": bb_params, "lr": 1e-4}) #사전 학습된 본체는 느리게
    return torch.optim.AdamW(param_groups, weight_decay=0.01)

optimizer = build_optimizer()

## 단계적 동결 해제

In [ ]:
stages = model.resnet.encoder.stages #모든 레이어 호출
def unfreeze_stage(idx: int):
    """
    4번째 레이어 : idx = -1
    3번째 레이어 : idx = -2
    2번째 레이어 : idx = -3

    후에 각 에포크별로 동결 해제 할 레이어를 idx로 지정
    """
    for p in stages[idx].parameters():
        p.requires_grad = True

def reinit_optimizer():
    """
    동결 해제 후 옵티마이저 재구성
    """
    global optimizer
    optimizer = build_optimizer()

## 학습

In [ ]:
import time
start_time = time.time() # 학습 시작 시간

epochs = 10
unfreeze_schedule = {4: -1, 7: -2}  # 에포크 4에서 4번째 레이어, 에포크 7에서 3번째 레이어 동결해제

for epoch in range(1, epochs + 1): #각 에포크마다 학습 진행

    #에포크에 맞추어 레이어 동결해제
    if epoch in unfreeze_schedule:
        unfreeze_stage(unfreeze_schedule[epoch]) #동결 해제
        reinit_optimizer() #옵티마이저 재구성

    model.train()  #모델 학습모드로 변환
    total, correct, loss_sum = 0, 0, 0.0 #train 이미지 수, 맞힌 개수, 손실 누적합
    for imgs, labels in train_loader: #각 배치별로
        imgs, labels = imgs.to(device), labels.to(device) #데이터를 GPU로 옮김

        optimizer.zero_grad(set_to_none=True) #이전 배치에서의 그레디언트를 초기화
        logits = model(pixel_values=imgs).logits #로짓 출력 결과값 얻음
        loss = criterion(logits, labels) #손실 계산
        loss.backward() #역전파 진행
        optimizer.step() #계산된 그레디언트로 모델의 파라미터를 한 스텝 업데이트

        loss_sum += loss.item() * labels.size(0) #이번 배치의 손실값을 누적합에 더함, lebels.size(0)는 배치의 크기
        correct += (logits.argmax(1) == labels).sum().item() #정답을 맞춘 것의 개수
        total += labels.size(0) #처리한 데이터의 개수 (배치 크기)

    #평균 손실과 정확도 계산
    train_loss = loss_sum / total
    train_acc  = correct / total * 100

    # validation 평가
    # train의 결과 평가와 같은 구조
    model.eval() #모델 평가모드로 전환
    v_total, v_correct, v_loss_sum = 0, 0, 0.0
    with torch.no_grad(): #평가에선 그레디언트를 사용하지 않음
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(pixel_values=imgs).logits

            v_loss_sum += criterion(logits, labels).item() * labels.size(0)
            v_correct += (logits.argmax(1) == labels).sum().item()
            v_total += labels.size(0)

    val_loss = v_loss_sum / v_total
    val_acc  = v_correct / v_total * 100
    #학습, 검증 손실 및 정확도 출력
    print(f"[{epoch}/{epochs}] train_loss {train_loss:.4f} acc {train_acc:.2f}% | val_loss {val_loss:.4f} acc {val_acc:.2f}%")

print("-"*30)
total_time = time.time() - start_time
print(f"Progressive Fine Tuning 학습 소요 시간: {total_time:.2f}초")

[1/10] train_loss 1.0468 acc 66.95% | val_loss 0.7626 acc 74.28%
[2/10] train_loss 0.7277 acc 75.51% | val_loss 0.6931 acc 76.21%
[3/10] train_loss 0.6772 acc 76.96% | val_loss 0.6617 acc 76.99%
[4/10] train_loss 0.4389 acc 84.82% | val_loss 0.3283 acc 88.45%
[5/10] train_loss 0.1284 acc 96.02% | val_loss 0.3371 acc 88.71%
[6/10] train_loss 0.0333 acc 99.28% | val_loss 0.3443 acc 89.60%
[7/10] train_loss 0.0646 acc 97.76% | val_loss 0.4448 acc 88.34%
[8/10] train_loss 0.0367 acc 98.77% | val_loss 0.4435 acc 89.36%
[9/10] train_loss 0.0220 acc 99.22% | val_loss 0.4309 acc 89.84%
[10/10] train_loss 0.0255 acc 99.13% | val_loss 0.5117 acc 88.87%
------------------------------
Progressive Fine Tuning 학습 소요 시간: 467.95초


## 테스트 평가

In [ ]:
#validation의 경우와 같음
model.eval()
total, correct, loss_sum = 0, 0, 0.0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        logits = model(pixel_values=imgs).logits
        loss_sum += criterion(logits, labels).item() * labels.size(0)
        correct += (logits.argmax(1) == labels).sum().item()
        total += labels.size(0)

test_loss = loss_sum / total
test_acc  = correct / total * 100
print("Progressive Fine Tuning CIFAR-10 테스트 손실 및 정확도")
print(f"test_loss {test_loss:.4f} acc {test_acc:.2f}%")

Progressive Fine Tuning CIFAR-10 테스트 손실 및 정확도
test_loss 0.5359 acc 88.88%
